In [1]:
a = {"a":2,"b":3}
a

{'a': 2, 'b': 3}

In [6]:
for i,fn in a.items():
    print(i)
    print(fn)

a
2
b
3


In [12]:
import numpy as np
x = np.random.rand(3)
y = x.astype(np.float32)
np.result_type(y)

dtype('float32')

In [27]:
N = int(1e8)
x = np.random.rand(N,6).astype(np.float32)
x.nbytes/10**6, (N*6*8)/10**6, "MB", (x.nbytes*300)/10**9,"GB"

(2400.0, 4800.0, 'MB', 720.0, 'GB')

In [38]:
def f_acceleration_flat(state, *, fn_acc):
    # Reshape 0:r,r_dot 1:planets 2:x,y
    Y = state.reshape(2, 3, 2)
    return np.array([Y[1], fn_acc(Y[0])]).flatten()

In [39]:
from functools import partial
# Implement using np arrays and broadcasting
def get_acceleration_numpy(X: np.ndarray) -> np.ndarray:
    vec_diff = X[:, np.newaxis] - X
    distance_matrix = np.linalg.norm(vec_diff, axis=2) ** 3

    # Set distance 0 to inf to avoid dividing by 0
    distance_matrix[distance_matrix == 0] = np.inf

    acceleration = vec_diff / distance_matrix[:, :, np.newaxis]

    return -np.sum(acceleration, axis=0)



partial(f_acceleration_flat,  fn_acc=get_acceleration_numpy)(np.random.rand(4,3))

array([ 3.67568330e-01,  5.72946167e-01,  7.40036896e-03,  1.78390026e-01,
        9.47619500e-01,  6.94870107e-01,  2.20183613e+01,  8.69117497e+00,
       -2.60834443e+01,  4.94801598e+00,  4.06508295e+00, -1.36391909e+01])

In [43]:
import scipy as sp
import numpy as np
from functools import partial

def f_acceleration_flat(x, y_flat, *, fn_acc):
    # Reshape 0:r,r_dot 1:planets 2:x,y
    Y = y_flat.reshape(2, 3, 2)
    return np.array([Y[1], fn_acc(Y[0])]).flatten()


def integrator_scipy(X, V, fn_acc, tmax, method="RK45"):
    phase_state = [X, V]
    Y_flat = np.array(phase_state).flatten()

    scipy_ivp = sp.integrate.solve_ivp(
        partial(f_acceleration_flat, fn_acc=fn_acc),
        (0, tmax),
        Y_flat,
        method=method,
        # t_eval=t_eval,
        vectorized=False,
        atol=1e-7,
        rtol=1e-7,
    )
    
    scipy_ivp_trajectory = scipy_ivp.y.reshape(2, 6, -1)[0].T.reshape(-1, 3, 2)
    scipy_ivp_velocity_trajectory = scipy_ivp.y.reshape(2, 6, -1)[1].T.reshape(-1, 3, 2)
    return [scipy_ivp_trajectory, scipy_ivp_velocity_trajectory]

In [44]:
r = np.array([[-1, 0], [1, 0], [0, 0]])


r_dot = np.array([[0.080584, 0.588836], [0.080584, 0.588836], [-0.161168, -1.177672]])


Xs, Vs = integrator_scipy(
    r, r_dot, fn_acc=get_acceleration_numpy, tmax=2
)